## Merge database tables into a flat dataframe

In [1]:
import sqlite3
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
pd.set_option("display.max_colwidth", 60)
pd.set_option("display.max_rows", 100)

In [2]:
RELEASE_DB_NAME = "/scratch/arjunth2001/princeton/release_db.sqlite"
conn = sqlite3.connect(RELEASE_DB_NAME)

### Read in the tables

In [3]:
# we'll merge everything into this dataframe
df = pd.read_sql_query("SELECT * FROM policy_snapshots", conn)

sites_df = pd.read_sql_query("SELECT * FROM sites", conn)
policy_texts_df = pd.read_sql_query("SELECT * FROM policy_texts", conn)
alexa_ranks_df = pd.read_sql_query("SELECT * FROM alexa_ranks", conn)

In [4]:
# Left join with policy text table
df = pd.merge(df, policy_texts_df, how="left", left_on="policy_text_id", right_on="id")

# Left join with sites table
df = pd.merge(df, sites_df, how="left", left_on="site_id", right_on="id")

# Left join with alexa ranks table
df = pd.merge(df, alexa_ranks_df, how="left", on=['site_id', 'year', 'phase'])

In [5]:
len([i for i in df.head(4)["policy_text"][0].split("\n") if len(i)!=0 and i!="" and len(i.split())>=15])

29

In [8]:
df2 = df.sample(10000)

In [11]:
import random
def get_sample(x):
    try:
        return random.choice([i for i in x.split("\n") if len(i)!=0 and i!="" and len(i.split())>=10])
    except:
        return [i for i in x.split("\n") if len(i)!=0 and i!=""][0]
                        
df2["sample"] = df2["policy_text"].progress_apply(lambda x:get_sample(x))

  0%|          | 0/10000 [00:00<?, ?it/s]

In [12]:
df2 = df2.reset_index()

In [13]:
df2.to_csv("annotate.csv")